In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
import cv2
import numpy as np
import pandas as pd
import os
import seaborn as sns
from functools import partial
sns.set_context('notebook')
import matplotlib.pyplot as plt



In [3]:
def hist_equal(img,L):
    '''takes image and an output space [0,L] as an input and gives an equalized image(in float) as output'''
    img = img.astype('float')
    epsilon = 1e-6 #a small number to remove divide by zero error
    if len(img.shape) == 2:
        return (L*(img - min(img.flatten()))/(max(img.flatten())-min(img.flatten())+epsilon))
    else:
        img[:,:,0] = L*(img[:,:,0] - min(img[:,:,0].flatten()))/(max(img[:,:,0].flatten())-min(img[:,:,0].flatten())+epsilon)
        img[:,:,1] = L*(img[:,:,1] - min(img[:,:,1].flatten()))/(max(img[:,:,1].flatten())-min(img[:,:,1].flatten())+epsilon)
        img[:,:,2] = L*(img[:,:,2] - min(img[:,:,2].flatten()))/(max(img[:,:,2].flatten())-min(img[:,:,2].flatten())+epsilon)
        return img

#### Load Directories and get file paths

In [43]:
my_dir = os.getcwd()  # grabs current work dir
all_file_paths = os.listdir()  # grabs all items in current work dir

data_path = [data for data in all_file_paths if 'traindata' in data.lower()]  # searches for data folder and retrieves

picture_folder_path = os.path.join(my_dir, data_path[0])  # joins data folder and current work dir to get picture path
picture_folder_path

all_pictures = os.listdir(picture_folder_path)  # list of pictures names in folder

picture_complete_path = [os.path.join(picture_folder_path, pic) for pic in
                         all_pictures]  # complete path to each picture

PP_path = os.path.join(my_dir, "TrainData_PP")  # PP: Post Processed



# split the picture path and return only the OG file name
file_names = [picture_complete_path[x].split('TrainData\\')[1] for x in range(len(picture_complete_path))]

['C:\\Users\\adamc\\Jupyter\\ECE-524-Neural_Networks\\Competition_Project\\TrainData\\019068.jpg', 'C:\\Users\\adamc\\Jupyter\\ECE-524-Neural_Networks\\Competition_Project\\TrainData\\019110.jpg', 'C:\\Users\\adamc\\Jupyter\\ECE-524-Neural_Networks\\Competition_Project\\TrainData\\019115.jpg', 'C:\\Users\\adamc\\Jupyter\\ECE-524-Neural_Networks\\Competition_Project\\TrainData\\019121.jpg', 'C:\\Users\\adamc\\Jupyter\\ECE-524-Neural_Networks\\Competition_Project\\TrainData\\019140.jpg', 'C:\\Users\\adamc\\Jupyter\\ECE-524-Neural_Networks\\Competition_Project\\TrainData\\019151.jpg', 'C:\\Users\\adamc\\Jupyter\\ECE-524-Neural_Networks\\Competition_Project\\TrainData\\019159.jpg', 'C:\\Users\\adamc\\Jupyter\\ECE-524-Neural_Networks\\Competition_Project\\TrainData\\019165.jpg', 'C:\\Users\\adamc\\Jupyter\\ECE-524-Neural_Networks\\Competition_Project\\TrainData\\019171.jpg']


#### Test Files, Load Test Images

In [ ]:
# img_1 = load_img(picture_complete_path[0])
# print("FIRST IMAGE DETAILS:", "\nClass: ", type(img_1), "\nFile Format: ", img_1.format,
#       "\nImage type: ", img_1.mode, '\nImage Size:', img_1.size)
# print()

# imag_1_open_cv = cv2.imread(picture_complete_path[0])  # read image using cv
# test_hist_img1 = hist_equal(imag_1_open_cv, 1)  # use equilizer function

# cv2.imshow('No_PostProcessing', imag_1_open_cv)
# cv2.imshow('Post_Processing', test_hist_img1)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [55]:
post_processed_files = []

for n, img in enumerate(picture_complete_path):
    load_temp_file = cv2.imread(img)
    post_processed_files.append(hist_equal(load_temp_file, 1))
    
#     cv2.imshow('Post_Processing', post_processed_files[n])
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

    # take [0 ,1]  -> scale to 255
    post_processed_files[n] = cv2.convertScaleAbs(post_processed_files[n], alpha=(255.0))
    cv2.imwrite(os.path.join(PP_path, file_names[n]), post_processed_files[n],[int(cv2.IMWRITE_JPEG_QUALITY), 100])


    

In [ ]:
# Load Data
# rgb_scale_photos = [load_img(photo, color_mode='rgb', target_size=(320, 240)) for photo in picture_complete_path] # RGB
# rgb_scale_photos = [load_img(photo, color_mode='rgb') for photo in picture_complete_path] # RGB


array_photo_list = [img_to_array(photo) for photo in rgb_scale_photos] # RGB

X_train_all = np.asarray(array_photo_list)

In [ ]:
# create data generator
datagen = ImageDataGenerator(rotation_range=45)

it = datagen.flow(np.expand_dims(X_train_all[0],0) ,batch_size=1)

In [ ]:
batch = it.next()
# convert to unsigned integers for viewing
image = batch[0].astype('uint8')
# plot raw pixel data
plt.imshow(image)